In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy import stats
import time
import datetime
import pandas as pd

import mysql.connector
from mysql.connector import Error

from bokeh.plotting import *
from bokeh.layouts import gridplot
from bokeh.models import *# Span, ColumnDataSource, LogColorMapper, ColorMapper, LogTicker, ColorBar, BasicTicker, LinearColorMapper, PrintfTickFormatter, HoverTool, CategoricalColorMapper, Range1d, Title
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import show, output_notebook, reset_output
output_notebook()
from bokeh.models.glyphs import Text
import bokeh.palettes as bp

# import hail as hl
import json
import urllib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

from pathlib import Path
from joblib import Parallel, delayed

Loading BokehJS ...

In [2]:
json_url = urllib.request.urlopen('https://api.helioviewer.org/?action=getDataSources')
hv_keys = json.loads(json_url.read())

In [3]:
hv_sid = pd.DataFrame(columns=['OBS','SOURCE_ID'])

# while sid=='sourceId':
#     key=hv_keys.keys()

for key1 in hv_keys.keys():
    for key2 in hv_keys[key1].keys():
        if 'sourceId' in hv_keys[key1][key2].keys(): 
            hv_sid.loc[len(hv_sid)] = " ".join([key1, key2]), hv_keys[key1][key2]['sourceId']
        else:
            for key3 in hv_keys[key1][key2].keys():
                if 'sourceId' in hv_keys[key1][key2][key3].keys(): 
                    hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3]), hv_keys[key1][key2][key3]['sourceId']
                else:
                    for key4 in hv_keys[key1][key2][key3].keys():
                        if 'sourceId' in hv_keys[key1][key2][key3][key4].keys(): 
                            hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3, key4]), hv_keys[key1][key2][key3][key4]['sourceId']
                        else:
                            for key5 in hv_keys[key1][key2][key3][key4].keys():
                                if 'sourceId' in hv_keys[key1][key2][key3][key4][key5].keys(): 
                                    hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3, key4, key5]), hv_keys[key1][key2][key3][key4][key5]['sourceId']
                                else:
                                    for key6 in hv_keys[key1][key2][key3][key4][key5].keys():
                                        if 'sourceId' in hv_keys[key1][key2][key3][key4][key5][key6].keys(): 
                                            hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3, key4, key5,key6]), hv_keys[key1][key2][key3][key4][key5][key6]['sourceId']    

In [4]:
hv_sid = hv_sid.sort_values(['SOURCE_ID']).reset_index(drop=True)
hv_sid

,OBS,SOURCE_ID
0,SOHO EIT 171,0
1,SOHO EIT 195,1
2,SOHO EIT 284,2
3,SOHO EIT 304,3
4,SOHO LASCO C2 white-light,4
...,...,...
72,Hinode XRT Al_poly Any,10009
73,Hinode XRT Be_med Any,10010
74,Hinode XRT Be_thin Any,10011
75,Hinode XRT C_poly Any,10012


In [8]:
from hv_setup import *

In [9]:
def sql_hv(sourceId, obs=None):
    query = "SELECT date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) WHERE sourceId={} GROUP BY date_format(date, '%Y-%m-%d 00:00:00');".format(sourceId)
    hv = sql_query(query)
    return hv_prepare(hv, sourceId, obs)

par = Parallel(n_jobs=20)
start_time=time.time()
results = par(delayed(sql_hv)(df['SOURCE_ID'], df['OBS']) for ind, df in hv_sid.iterrows())
print(time.time()-start_time)

68.48031854629517


In [10]:
hv_obs = {}
for i in range(len(hv_sid)):
    if results[i].empty:
        hv_sid = hv_sid.drop(index = hv_sid.index[hv_sid.index == i])
        continue
    hv_obs[hv_sid.iloc[i]['SOURCE_ID']] = results[i]
hv_obs[np.random.choice(list(hv_obs.keys()))]

,count,date,Year,Day,SOURCE_ID,OBS
0,NaN,2006-12-01,2006 December,1,45,Hinode XRT Al_poly Al_thick
1,NaN,2006-12-02,2006 December,2,45,Hinode XRT Al_poly Al_thick
2,NaN,2006-12-03,2006 December,3,45,Hinode XRT Al_poly Al_thick
3,NaN,2006-12-04,2006 December,4,45,Hinode XRT Al_poly Al_thick
4,NaN,2006-12-05,2006 December,5,45,Hinode XRT Al_poly Al_thick
...,...,...,...,...,...,...
3952,NaN,2017-09-26,2017 September,26,45,Hinode XRT Al_poly Al_thick
3953,NaN,2017-09-27,2017 September,27,45,Hinode XRT Al_poly Al_thick
3954,NaN,2017-09-28,2017 September,28,45,Hinode XRT Al_poly Al_thick
3955,NaN,2017-09-29,2017 September,29,45,Hinode XRT Al_poly Al_thick


In [ ]:
# for ind, df in hv_sid[16:17].iterrows():
#     start_time=time.time()
#     print('INFO:', ind, df['SOURCE_ID'])
#     results = [sql_query(df['SOURCE_ID'])]
#     print(time.time()-start_time)

# Coverage

In [ ]:
for observatory in hv_keys.keys():
    panels_obs=[]
    for ind, df_obs in hv_sid[hv_sid['OBS'].str.match(observatory)].iterrows():
        
        df = hv_obs[df_obs['SOURCE_ID']].copy()
        sid = df['SOURCE_ID'].unique()[0]
        name = df['OBS'][0]
        name_ = name.replace(" ", "_")

        years = np.array(df['Year'].unique()).astype(str)# hv_cov.index.values#.astype(str)
        days = df['Day'].unique().astype(str) # np.arange(1,32).astype(str)

        colors = bp.Viridis[256]# ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]

        TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

        # output_file('AIA1600_coverage.html')
        panels = []
        for mapper_type, mapper, ticker in zip(["log", "linear"],
                                               [LogColorMapper, LinearColorMapper],
                                               [LogTicker, BasicTicker]):
            p = figure(#x_axis_type='datetime',
                       x_range=years, y_range=list(reversed(days)),
                       x_axis_location="above", sizing_mode='stretch_both',# width_policy='max', height_policy='max',#, plot_width=1400,
                       x_axis_label="Year Month", y_axis_label="Date",
                       tools=TOOLS,
                       tooltips=[('Date', '@Year @Day'), ('#Data Files', '@count')])
                       

            total_days = (hv_obs[sid]['count']>=0).sum()
            total_files = (hv_obs[sid]['count']).sum()
            
            p.add_layout(Title(text="%s Coverage"%(name), text_font_size='14pt'), 'above')
            p.add_layout(Title(text="Date Range: %s - %s"%(df.dropna()['date'].min().strftime("%Y, %b %d"), df.dropna()['date'].max().strftime("%Y, %b %d"))), 'above')
            p.add_layout(Title(text="Total Files: %d | Total Days: %d | Source ID: %d"%(total_files, total_days, df['SOURCE_ID'].unique()[0]), text_font_style="italic"), 'above')
#             p.add_layout(Title(text="Total Days: %d"%total_days, text_font_style="italic"), 'above')
#             p.add_layout(Title(text="Source ID: %d"%df['SOURCE_ID'].unique()[0], text_font_style="italic"), 'above')

            # p.grid.grid_line_color = None
            p.axis.axis_line_color = None
            p.axis.major_tick_line_color = None
            p.axis.major_label_text_font_size = "7px"
            p.axis.major_label_standoff = 0
            p.xaxis.major_label_orientation = np.pi / 3
            p.xaxis.axis_label_text_font_size = "12pt"
            p.yaxis.axis_label_text_font_size = "12pt"
            p.xaxis.visible = True
            p.xgrid.visible = True
            p.ygrid.visible = False

            p.xaxis.major_label_text_font_size = "7pt"
            p.yaxis.major_label_text_font_size = "8pt"


            p.rect(x="Year", y="Day", width=1, height=1,
                   source=df, hover_fill_alpha=0.3,
                   fill_color={'field': 'count', 'transform': mapper(palette=colors, low=0.1, high=np.nanmax(df['count']))},
                   line_color=None, dilate=True)
            
            num_ticks=10
            if (len(df[df['count']>0]['count'].unique()) <= 10):
                num_ticks = len(df[df['count']>0]['count'].unique())
            color_bar = ColorBar(color_mapper = mapper(palette=colors, low=0.1, high=np.nanmax(df['count'])), 
                                 major_label_text_font_size="10px",
                                 ticker=ticker(desired_num_ticks=num_ticks),
                                 formatter=PrintfTickFormatter(format="%d"),
                                 label_standoff=6, border_line_color=None, location=(0, 0))
            p.add_layout(color_bar, 'right')
#             p.width_policy = 'fit'
#             p.height_policy = 'fit'
            panel = Panel(child=p, title=mapper_type)
            panels.append(panel)
        tabs = Tabs(tabs=panels)
#         show(tabs)
        panel_obs = Panel(child=tabs, title=name.replace(observatory+' ',''))
        panels_obs.append(panel_obs)
    tabs_obs = Tabs(tabs=panels_obs)
#     show(tabs_obs)
    save(tabs_obs, filename='./coverages/%s_coverage.html'%observatory, title="Coverage plot for %s"%observatory)

In [ ]:
import numpy as np

from bokeh.core.properties import Instance, String
from bokeh.io import show
from bokeh.models import ColumnDataSource, LayoutDOM
from bokeh.util.compiler import TypeScript

TS_CODE = """
// This custom model wraps one part of the third-party vis.js library:
//
//     http://visjs.org/index.html
//
// Making it easy to hook up python data analytics tools (NumPy, SciPy,
// Pandas, etc.) to web presentations using the Bokeh server.

import {LayoutDOM, LayoutDOMView} from "models/layouts/layout_dom"
import {ColumnDataSource} from "models/sources/column_data_source"
import {LayoutItem} from "core/layout"
import * as p from "core/properties"

declare namespace vis {
  class Graph3d {
    constructor(el: HTMLElement, data: object, OPTIONS: object)
    setData(data: vis.DataSet): void
  }

  class DataSet {
    add(data: unknown): void
  }
}

// This defines some default options for the Graph3d feature of vis.js
// See: http://visjs.org/graph3d_examples.html for more details.
const OPTIONS = {
  width: '600px',
  height: '600px',
  style: 'surface',
  showPerspective: true,
  showGrid: true,
  keepAspectRatio: true,
  verticalRatio: 1.0,
  legendLabel: 'stuff',
  cameraPosition: {
    horizontal: -0.35,
    vertical: 0.22,
    distance: 1.8,
  },
}
// To create custom model extensions that will render on to the HTML canvas
// or into the DOM, we must create a View subclass for the model.
//
// In this case we will subclass from the existing BokehJS ``LayoutDOMView``
export class Surface3dView extends LayoutDOMView {
  model: Surface3d

  private _graph: vis.Graph3d

  initialize(): void {
    super.initialize()

    const url = "https://cdnjs.cloudflare.com/ajax/libs/vis/4.16.1/vis.min.js"
    const script = document.createElement("script")
    script.onload = () => this._init()
    script.async = false
    script.src = url
    document.head.appendChild(script)
  }

  private _init(): void {
    // Create a new Graph3s using the vis.js API. This assumes the vis.js has
    // already been loaded (e.g. in a custom app template). In the future Bokeh
    // models will be able to specify and load external scripts automatically.
    //
    // BokehJS Views create <div> elements by default, accessible as this.el.
    // Many Bokeh views ignore this default <div>, and instead do things like
    // draw to the HTML canvas. In this case though, we use the <div> to attach
    // a Graph3d to the DOM.
    this._graph = new vis.Graph3d(this.el, this.get_data(), OPTIONS)

    // Set a listener so that when the Bokeh data source has a change
    // event, we can process the new data
    this.connect(this.model.data_source.change, () => {
      this._graph.setData(this.get_data())
    })
  }

  // This is the callback executed when the Bokeh data has an change. Its basic
  // function is to adapt the Bokeh data source to the vis.js DataSet format.
  get_data(): vis.DataSet {
    const data = new vis.DataSet()
    const source = this.model.data_source
    for (let i = 0; i < source.get_length()!; i++) {
      data.add({
        x: source.data[this.model.x][i],
        y: source.data[this.model.y][i],
        z: source.data[this.model.z][i],
      })
    }
    return data
  }

  get child_models(): LayoutDOM[] {
    return []
  }

  _update_layout(): void {
    this.layout = new LayoutItem()
    this.layout.set_sizing(this.box_sizing())
  }
}

// We must also create a corresponding JavaScript BokehJS model subclass to
// correspond to the python Bokeh model subclass. In this case, since we want
// an element that can position itself in the DOM according to a Bokeh layout,
// we subclass from ``LayoutDOM``
export namespace Surface3d {
  export type Attrs = p.AttrsOf<Props>

  export type Props = LayoutDOM.Props & {
    x: p.Property<string>
    y: p.Property<string>
    z: p.Property<string>
    data_source: p.Property<ColumnDataSource>
  }
}

export interface Surface3d extends Surface3d.Attrs {}

export class Surface3d extends LayoutDOM {
  properties: Surface3d.Props
  __view_type__: Surface3dView

  constructor(attrs?: Partial<Surface3d.Attrs>) {
    super(attrs)
  }

  // The ``__name__`` class attribute should generally match exactly the name
  // of the corresponding Python class. Note that if using TypeScript, this
  // will be automatically filled in during compilation, so except in some
  // special cases, this shouldn't be generally included manually, to avoid
  // typos, which would prohibit serialization/deserialization of this model.
  static __name__ = "Surface3d"

  static init_Surface3d() {
    // This is usually boilerplate. In some cases there may not be a view.
    this.prototype.default_view = Surface3dView

    // The @define block adds corresponding "properties" to the JS model. These
    // should basically line up 1-1 with the Python model class. Most property
    // types have counterparts, e.g. ``bokeh.core.properties.String`` will be
    // ``p.String`` in the JS implementatin. Where the JS type system is not yet
    // as rich, you can use ``p.Any`` as a "wildcard" property type.
    this.define<Surface3d.Props>({
      x:            [ p.String   ],
      y:            [ p.String   ],
      z:            [ p.String   ],
      data_source:  [ p.Instance ],
    })
  }
}
"""

# This custom extension model will have a DOM view that should layout-able in
# Bokeh layouts, so use ``LayoutDOM`` as the base class. If you wanted to create
# a custom tool, you could inherit from ``Tool``, or from ``Glyph`` if you
# wanted to create a custom glyph, etc.
class Surface3d(LayoutDOM):

    # The special class attribute ``__implementation__`` should contain a string
    # of JavaScript code that implements the browser side of the extension model.
    __implementation__ = TypeScript(TS_CODE)

    # Below are all the "properties" for this model. Bokeh properties are
    # class attributes that define the fields (and their types) that can be
    # communicated automatically between Python and the browser. Properties
    # also support type validation. More information about properties in
    # can be found here:
    #
    #    https://docs.bokeh.org/en/latest/docs/reference/core/properties.html#bokeh-core-properties

    # This is a Bokeh ColumnDataSource that can be updated in the Bokeh
    # server by Python code
    data_source = Instance(ColumnDataSource)

    # The vis.js library that we are wrapping expects data for x, y, and z.
    # The data will actually be stored in the ColumnDataSource, but these
    # properties let us specify the *name* of the column that should be
    # used for each field.
    x = String

    y = String

    z = String


x = np.arange(0, 300, 10)
y = np.arange(0, 300, 10)
xx, yy = np.meshgrid(x, y)
xx = xx.ravel()
yy = yy.ravel()
value = np.sin(xx / 50) * np.cos(yy / 50) * 50 + 50

source = ColumnDataSource(data=dict(x=xx, y=yy, z=value))

surface = Surface3d(x="x", y="y", z="z", data_source=source, width=600, height=600)

show(surface)

# Histogram

In [ ]:
def bin_width(m):
    n = np.int(np.log10(m+1))
    n = 10**(n-1)
    q = np.ceil(m/(n*(36))).astype(int)
    bw = max(q*n,1)
    return bw#, m//n+1

In [ ]:
for observatory in hv_keys.keys():
    panels_obs=[]
    for ind, df_obs in hv_sid[hv_sid['OBS'].str.match(observatory)].iterrows():
        
        df = hv_obs[df_obs['SOURCE_ID']].copy()
        sid = df['SOURCE_ID'].unique()[0]
        name = df['OBS'][0]
        name_ = name.replace(" ", "_")

        df = hv_obs[sid].copy()
        df = df.dropna().reset_index(drop=True)

        name = df['OBS'].unique()[0]
        name_ = name.replace(" ", "_")

        bin_size = bin_width(df['count'].max())# np.arange(0,count.max(),) 30#.astype(int)#100
        btabs = interactive_histogram(df['count'], sid, name, bin_size)
        counts = df['count']
        title=name

        arr_hist, edges, patches = plt.hist(counts, bins=np.arange(0, counts.max()+bin_size, bin_size))
        cum_bin_size  = max(bin_size//10,1)
        cum_hist, cum_edges, patches = plt.hist(counts, bins=np.arange(0,counts.max()+cum_bin_size, cum_bin_size), cumulative=True)
        plt.close()

        # Column data source
        df_hist = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
        total = df_hist['count'].sum()
        df_hist['f_count'] = ['%d' % count for count in df_hist['count']]
        df_hist['f_percent'] = ['%.2f%%' %(count/total*100) for count in df_hist['count']]
        df_hist['f_interval'] = ['[%d,%d) ' % (left, right) for left, right in zip(df_hist['left'], df_hist['right'])]
        # column data source
        hist_src = ColumnDataSource(df_hist)

        #cumulative data
        cumulative_data = cum_hist#np.cumsum(arr_hist)
        x_bins = cum_edges[1:]#edges[1:]# np.arange(0, counts.max(), bin_size)[1:]
        df_cum = pd.DataFrame({'count_cum': cumulative_data, 'x': x_bins})
        cum_src = ColumnDataSource(df_cum)
    #     df_hist['f_count'] = np.log10(df_hist['f_count']+1)
        # Set up the figure same as before
        panels = []

        for axis_type in ["log","linear"]:
            p = figure(y_axis_type = axis_type,
                       x_axis_label = 'No. of Data files', y_axis_label = 'Day count', 
                       background_fill_color="#fafafa",
                       y_range = (0.9, df_hist['count'].max() + df_hist['count'].max()//10))

            # Add a quad glyph with source this time
            p.quad(bottom=0.9, top='count', left='left', right='right', source=hist_src, fill_color='navy', alpha=0.5,
                   hover_fill_color='navy', hover_fill_alpha=0.2, line_color='white', legend='Histogram')
    #         p.y_range(Range1d(0.8,df_hist['count'].max()))
            # Add style to the plot
            p.title.align = 'center'
            p.title.text_font_size = '18pt'
            p.xaxis.axis_label_text_font_size = '12pt'
            p.xaxis.major_label_text_font_size = '12pt'
            p.yaxis.axis_label_text_font_size = '12pt'
            p.yaxis.major_label_text_font_size = '12pt'
            p.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
            p.xaxis[0].formatter = NumeralTickFormatter(format='0,0')
            
            df_stats = pd.DataFrame({'height': np.linspace(0.5, df_hist['count'].max(), 2),
                                     'mean':np.nanmean(counts), 'median': np.nanmedian(counts), 'mode':stats.mode(counts)[0][0]})
            p.line(x='mean', y='height', line_color="black", line_dash='solid', line_width = 4, legend="Mean (%.2f)"%(df_stats['mean'][0]), source=df_stats)
            p.line(x='median', y='height', line_color = "red", line_dash='dashed', line_width=3, legend="Median (%.2f)"%(df_stats['median'][0]), source=df_stats)
#             p.line(x='mode', y='height', line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode (%.2f)"%(df_stats['mode'][0]), source=df_stats)

            total_days = (counts>=0).sum()
            total_files = counts.sum()
            
            p.add_layout(Title(text = "Histogram for %s"%title, text_font_size = "16pt", text_font_style="bold"), 
                         place = 'above')
            p.add_layout(Title(text="Date range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))),
                         place = 'above')
            p.add_layout(Title(text="Total Files: {:,.0f} | Total Days: {:,.0f} | Source ID: {}".format(total_files, total_days, sid), text_font_style="italic"),
                         place = 'above')

    #         p.legend.location = "top_left"
    #         p.grid.grid_line_color="white"

    #         text_source = ColumnDataSource(dict(x=[x_bins.max()*3/4],y=[df_hist['count'].max()*3/4],text=['Total Day Count = \n %d'%total]))
    #         glyph = Text(x="x", y="y", text="text", text_color="black")
    #         p.add_glyph(text_source, glyph)

            # Add a hover tool referring to the formatted columns
            hover = HoverTool(tooltips = [('#Data files', '@f_interval'),
                                          ('Day count', '@f_count{0,0}'),
                                          ('Day count percentage', '@f_percent')],
                              mode= 'vline')

            # Add the hover tool to the graph
            p.add_tools(hover)


            p2 = figure(y_axis_type=axis_type,
                       x_axis_label = 'No. of Data files', 
                       y_axis_label = 'Day count',
                       background_fill_color="#fafafa")
                       

            p2_line = p2.line(x='x', y='count_cum', line_color='#036564', line_width=3, source=cum_src, legend="Cumulative distribution")
    #         p2_circle = p2.circle(x='x', y='count_cum', line_color='#036564', line_width=5, source=cum_src, hover_line_alpha=0.5, legend="Cumulative distribution" )
            p2.add_layout(Title(text = "Cumulative Distribution for %s"%title, text_font_size = "16pt", text_font_style="bold"), place = 'above')
            p2.add_layout(Title(text="Date range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 'above')
            p2.add_layout(Title(text="Total Files: {:,.0f} | Total Days: {:,.0f} | Source ID: {}".format(total_files, total_days, sid), text_font_style="italic"), 'above')
#             p2.add_layout(Title(text="Total Days: %d"%total_days, text_font_style="italic"), 'above')
#             p2.add_layout(Title(text="Source ID: %d"%sid, text_font_style="italic"), 'above')
            
            hover = HoverTool(line_policy='nearest', 
                              tooltips = [('#Data files', '<@x{0,0}'), 
                                          ('Cumulative Day count', '@count_cum{0,0}')], 
                              mode='vline')
            df_cumstats = pd.DataFrame({'height': np.linspace(df_cum['count_cum'].min(),df_cum['count_cum'].max(),2),
                                        'mean':np.nanmean(counts), 'median': np.nanmedian(counts), 'mode':stats.mode(counts)[0][0]})
            p2.line(x='mean', y='height', line_color="black", line_dash='solid', line_width = 4, legend="Mean (%.2f)"%(df_cumstats['mean'][0]), source=df_cumstats)
            p2.line(x='median', y='height', line_color = "red", line_dash='dashed', line_width=3, legend="Median (%.2f)"%(df_cumstats['median'][0]), source=df_cumstats)
#             p2.line(x='mode', y='height', line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode (%.2f)"%(df_cumstats['mode'][0]), source=df_cumstats)
        

            # Add the hover tool to the graph
            p2.add_tools(hover)
            p2.title.align = 'center'
            p2.title.text_font_size = '18pt'
            p2.xaxis.axis_label_text_font_size = '12pt'
            p2.xaxis.major_label_text_font_size = '12pt'
            p2.yaxis.axis_label_text_font_size = '12pt'
            p2.yaxis.major_label_text_font_size = '12pt'
            p2.legend.location = "bottom_right"
            p2.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
            p2.xaxis[0].formatter = NumeralTickFormatter(format='0,0')

            grid = gridplot([[p, p2]], sizing_mode='stretch_both')# width_policy='max', height_policy='max')#,plot_width=1200, plot_height=1000, sizing_mode='scale_width')#, plot_width=250, plot_height=250)
            panel = Panel(child=grid, title=axis_type)
            panels.append(panel)
        tabs = Tabs(tabs=panels)
#         show(tabs)
        panel_obs = Panel(child=tabs, title=name.replace(observatory+' ',''))
        panels_obs.append(panel_obs)
    tabs_obs = Tabs(tabs=panels_obs)
#     show(tabs_obs)
    save(tabs_obs, filename='./histograms/%s_histogram.html'%observatory, title="Histogram and Cumulative distribution for %s"%observatory)

In [103]:
np.arctan(np.sqrt(3)-2)*180/np.pi

-15.000000000000005

In [ ]:
def interactive_histogram(counts, sid, title, bin_size=None):
    
    arr_hist, edges, patches = plt.hist(counts, bins=np.arange(0, counts.max()+bin_size, bin_size))
    cum_bin_size  = max(bin_size//10,1)
    cum_hist, cum_edges, patches = plt.hist(counts, bins=np.arange(0,counts.max()+cum_bin_size, cum_bin_size), cumulative=True)
    plt.close()
    
    # Column data source
    df_hist = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
    total = df_hist['count'].sum()
    df_hist['f_count'] = ['%d' % count for count in df_hist['count']]
    df_hist['f_percent'] = ['%.2f%%' %(count/total*100) for count in df_hist['count']]
    df_hist['f_interval'] = ['[%d,%d) ' % (left, right) for left, right in zip(df_hist['left'], df_hist['right'])]
    # column data source
    hist_src = ColumnDataSource(df_hist)
    
    #cumulative data
    cumulative_data = cum_hist#np.cumsum(arr_hist)
    x_bins = cum_edges[1:]#edges[1:]# np.arange(0, counts.max(), bin_size)[1:]
    df_cum = pd.DataFrame({'count_cum': cumulative_data, 'x': x_bins})
    cum_src = ColumnDataSource(df_cum)
#     df_hist['f_count'] = np.log10(df_hist['f_count']+1)
    # Set up the figure same as before
    panels = []
    
    for axis_type in ["log","linear"]:
        p = figure(y_axis_type = axis_type,
                   title = "Histogram for %s"%title,
                   x_axis_label = 'No. of Data files', 
                   y_axis_label = 'Day count',
                   background_fill_color="#fafafa",
                   y_range = (0.9, df_hist['count'].max() + df_hist['count'].max()//10))
        
        # Add a quad glyph with source this time
        p.quad(bottom=0.9, top='count', left='left', right='right', source=hist_src, fill_color='navy', alpha=0.5,
               hover_fill_color='navy', hover_fill_alpha=0.2, line_color='white', legend='Histogram')
#         p.y_range(Range1d(0.8,df_hist['count'].max()))
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
#         p.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        p.line(x=np.mean(counts), y=df_hist['count'], line_color="black", line_dash='solid', line_width = 4, legend="Mean # Data files")
        p.line(x=np.median(counts), y=df_hist['count'], line_color = "red", line_dash='dashed', line_width=3, legend="Median # Data files")
        p.line(x=stats.mode(counts)[0][0], y=df_hist['count'], line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode # Data files")
        
        total_days = (counts>=0).sum()
        total_files = counts.sum()
        
        p.add_layout(Title(text="Total Files: %d"%total_files, text_font_style="italic"), 'above')
        p.add_layout(Title(text="Total Days: %d"%total_days, text_font_style="italic"), 'above')
        p.add_layout(Title(text="Source ID: %d"%sid, text_font_style="italic"), 'above')
        
#         p.legend.location = "top_left"
#         p.grid.grid_line_color="white"
        
#         text_source = ColumnDataSource(dict(x=[x_bins.max()*3/4],y=[df_hist['count'].max()*3/4],text=['Total Day Count = \n %d'%total]))
#         glyph = Text(x="x", y="y", text="text", text_color="black")
#         p.add_glyph(text_source, glyph)

        # Add a hover tool referring to the formatted columns
        hover = HoverTool(tooltips = [('#Data files', '@f_interval'),
                                      ('Day count', '@f_count'),
                                      ('Day count percentage', '@f_percent')],
                          mode= 'vline')

        # Add the hover tool to the graph
        p.add_tools(hover)
        
        
        p2 = figure(y_axis_type=axis_type,
                   title = "Cumulative distribution for %s"%title,
                   x_axis_label = 'No. of Data files', 
                   y_axis_label = 'Day count',
                   background_fill_color="#fafafa")
        
        p2_line = p2.line(x='x', y='count_cum', line_color='#036564', line_width=3, source=cum_src, legend="Cumulative distribution")
#         p2_circle = p2.circle(x='x', y='count_cum', line_color='#036564', line_width=5, source=cum_src, hover_line_alpha=0.5, legend="Cumulative distribution" )
        hover = HoverTool(line_policy='nearest', tooltips = [('#Data files', '<@x'), ('Cumulative Day count', '@count_cum')], mode='vline')
        
#         p2.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        p2.line(x=np.mean(counts), y=df_cum['count_cum'], line_color="black", line_dash='solid', line_width = 4, legend="Mean # Data files")
        p2.line(x=np.median(counts), y=df_cum['count_cum'], line_color = "red", line_dash='dashed', line_width=3, legend="Median # Data files")
        p2.line(x=stats.mode(counts)[0][0], y=df_cum['count_cum'], line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode # Data files")
    
        # Add the hover tool to the graph
        p2.add_tools(hover)
        p2.title.align = 'center'
        p2.title.text_font_size = '18pt'
        p2.xaxis.axis_label_text_font_size = '12pt'
        p2.xaxis.major_label_text_font_size = '12pt'
        p2.yaxis.axis_label_text_font_size = '12pt'
        p2.yaxis.major_label_text_font_size = '12pt'
        p2.legend.location = "top_left"
        
        
        grid = gridplot([[p, p2]])#, plot_width=250, plot_height=250)
        panel = Panel(child=grid, title=axis_type)
        panels.append(panel)
    
    tabs = Tabs(tabs=panels)
    return tabs

In [ ]:
for sid in hv_obs.keys()
    df = hv_obs[sid].copy()
    name = df['OBS'].unique()[0]
    name_ = name.replace(" ", "_")
    
    df = df.dropna().reset_index(drop=True)
    f = open("./csv_files/%d_%s.csv"%(sid, name_), mode='w')
    f.write("# %s\n"%name)
    f.write("# %d\n"%sid)
    df.to_csv(f, columns=["date", "count", "SOURCE_ID", "OBS"], index=False)
    f.close()

In [ ]:
for key in hv_obs.keys():
    if len(hv_obs[key]) < 100: 
        plt.scatter(key, len(hv_obs[key].dropna().reset_index(drop=True)))
        print(key)